# Setup

set working directory to guitarist-net and add to path

In [1]:
%cd /content/drive/Othercomputers/My MacBook Pro/guitarist-net

/content/drive/Othercomputers/My MacBook Pro/guitarist-net


install requirements

In [2]:
!pip install -r requirements.txt &> /dev/null

imports

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import librosa
import librosa.display
import ui
from plot_listen.listen import play
from save_load import load_weights_from_file
from save_load import save_audio
from util import torch_to_numpy
from globals import *
from data.guitarset_loader import GuitarSetLoader
from ddsp.ddsp_hex_synth import DDSPHexSynth

# Load Trained Model

create model

In [ ]:
synth = DDSPHexSynth()

load weights, move to GPU, set to eval mode

In [ ]:
synth.load_state_dict(load_weights_from_file())
synth = synth.to(DEVICE)
synth.eval();

Type a number to select an option:
	0)	2022-07-08 08:08PM DDSP Mono Synth : training test
	1)	2022-07-09 01:40PM DDSP Mono Synth : load model and continue training
	2)	2022-07-09 11:45PM DDSP Mono Synth : resume training (third time)
	3)	2022-07-10 10:44AM DDSP Mono Synth : train (4th time)
	4)	2022-07-19 04:05PM DDSP Mono Synth Improved : synth_improvement
	5)	2022-07-19 06:56PM DDSP Mono Synth : train with loudness centered and 32 osc + 32 filter coeffs
	6)	2022-07-20 10:48AM DDSP Mono Synth : train low E
	7)	2022-07-20 12:24PM DDSP Mono Synth Improved : improve 32 param synth - e string
	8)	2022-07-20 07:03PM DDSP Mono Synth : train with lr scheduler
	9)	2022-07-22 11:34AM DDSP Mono Synth : train low E for a while
	10)	2022-07-25 06:42PM DDSP Mono Synth : train with Leaky ReLU
	11)	2022-07-26 04:41PM DDSP Mono Synth : try training with test dataset (one guitarist)
	12)	2022-07-27 06:09PM DDSP Mono Synth : folder deleting, harmonic + noise saving
	13)	2022-07-27 06:12PM DDSP Mono Syn

KeyboardInterrupt: ignored

# Load Track Data

In [ ]:
loader = GuitarSetLoader()
track_id = ui.list_selection_menu(loader.guitarset.track_ids)
track_data = loader.get_6_string_data_from_track_unchunked(track_id)

Type a number to select an option:
	0)	00_BN1-129-Eb_comp
	1)	00_BN1-129-Eb_solo
	2)	00_BN1-147-Gb_comp
	3)	00_BN1-147-Gb_solo
	4)	00_BN2-131-B_comp
	5)	00_BN2-131-B_solo
	6)	00_BN2-166-Ab_comp
	7)	00_BN2-166-Ab_solo
	8)	00_BN3-119-G_comp
	9)	00_BN3-119-G_solo
	10)	00_BN3-154-E_comp
	11)	00_BN3-154-E_solo
	12)	00_Funk1-114-Ab_comp
	13)	00_Funk1-114-Ab_solo
	14)	00_Funk1-97-C_comp
	15)	00_Funk1-97-C_solo
	16)	00_Funk2-108-Eb_comp
	17)	00_Funk2-108-Eb_solo
	18)	00_Funk2-119-G_comp
	19)	00_Funk2-119-G_solo
	20)	00_Funk3-112-C#_comp
	21)	00_Funk3-112-C#_solo
	22)	00_Funk3-98-A_comp
	23)	00_Funk3-98-A_solo
	24)	00_Jazz1-130-D_comp
	25)	00_Jazz1-130-D_solo
	26)	00_Jazz1-200-B_comp
	27)	00_Jazz1-200-B_solo
	28)	00_Jazz2-110-Bb_comp
	29)	00_Jazz2-110-Bb_solo
	30)	00_Jazz2-187-F#_comp
	31)	00_Jazz2-187-F#_solo
	32)	00_Jazz3-137-Eb_comp
	33)	00_Jazz3-137-Eb_solo
	34)	00_Jazz3-150-C_comp
	35)	00_Jazz3-150-C_solo
	36)	00_Rock1-130-A_comp
	37)	00_Rock1-130-A_solo
	38)	00_Rock1-90-C#_comp
	39)	00_Ro

# Create Input List

In [ ]:
inputs = []

for string_let in GUITAR_STRING_LETTERS:
    inputs.append(torch.tensor(track_data[f"{string_let}_f0"], dtype=DEFAULT_TORCH_DTYPE).to(DEVICE).unsqueeze(0))
    inputs.append(torch.tensor(track_data[f"{string_let}_loudness"], dtype=DEFAULT_TORCH_DTYPE).to(DEVICE).unsqueeze(0))

# Feed F0s and Loudnesses into Synth Model

In [ ]:
output = synth(*inputs)

# Listen to output

In [ ]:
play(output['reverb_audio'][0])

# Compare against original

In [ ]:
play(track_data['mic_audio'])

# Save off example

In [ ]:
# save_audio(output['reverb_audio'][0], f"/content/drive/MyDrive/Research/guitarist-net-storage/demos/hex_output_{track_id}.wav")